In [ ]:
import numpy as np
from matplotlib import pyplot as plt
#%matplotlib notebook
%matplotlib inline
import datetime as dt
from matplotlib import dates as mdates
import matplotlib
import pandas as pd
import corotools as ct
from IPython.display import display, Markdown

import os


population_file = os.environ['COVID19_SCRIPT_POPFILE']
jhu_data_dir = os.environ['COVID19_JHU_DIR']


import warnings
warnings.filterwarnings('ignore')

In [ ]:

pops = ct.get_populations('~/Daten/worldpop.csv')
MDlist = ct.get_jhu_data(datadir = jhu_data_dir)
Md = MDlist[0].copy()
lastday = dt.datetime(*[int(x) for x in Md['US'][-1,0:3]])
lastdayasstr = lastday.strftime("%a, %d.%m.%Y")

metat = ct.get_metatable(MDlist, pops, [x for x in Md.keys()], ignorelist=ct.list_continents)
mt, mt2, mt4 = ct.get_metatable124(MDlist, pops, [x for x in Md.keys()], ignorelist=ct.list_continents)
speciallist = []#['World','North America','South America','Europe','Asia','Africa']
speciallist.sort()
speciallist.append('Austria')
speciallist.append('Czechia')
speciallist.append('Germany')
speciallist.append('Iran')
speciallist.append('Israel')
speciallist.append('Sweden')
speciallist.append('Switzerland')
speciallist.append('Turkey')


plotsw = True




sources for data
* Johns-Hopkins-University [JHU github](https://github.com/CSSEGISandData/COVID-19)
* [worldometers](https://www.worldometers.info) (populations)

remarks:
* due to strong daily fluctuations, my stats concentrate on weekly numbers/changes
* some numbers from JHU are different to 'officially' reported ones (e.g. France) - take it with a grain of salt

scripts for evaluation this data and creating this page:

[xmhk/corotools](https://github.com/xmhk/corotools)

# tables: cases

remarks:
* 1st column: rank (-2 weeks, -4 weeks)
* **c(k)**: total cases (in thousand)
* **dc(k)**: last weeks increase in cases (in thousand)
* **c /1k**: total cases per population (per thousand)
* **dc /1m**: last weeks increase of cases per population (per million)
* **r(%)**: ratechange - relative change of cases last week vs. the week before

In [ ]:
tablesnum = 15
tablesmininf = 2000
allkeys=[]


cases_table = lambda sk :  ct.get_rank_table_from_df2(
        mt, mt2, mt4, sk,
        ['cases', 'cases_delta','cases_perpop', 'cases_delta_perpop', 'cases_ratechange'], 
        [1/1000,1/1000,1e3, 1e6,100],
        columnalias=['c(k)','dc(k)','c /1k', 'dc /1m', 'r(%)'],
         ascending=False, mininf=tablesmininf,
         num=tablesnum)

for sk in ['cases', 'cases_delta', 'cases_perpop','cases_delta_perpop', 'cases_ratechange']:
    display(Markdown("## %s week ending %s"%(sk,lastdayasstr)))
    if sk == 'cases_ratechange':
        display(Markdown("remark: countries which had below %.0f cases two weeks ago are ranked at 200"%tablesmininf))
    display(Markdown(cases_table(sk)))
    ll = ct.sort_df_and_get_keys(mt[mt['cases']>tablesmininf],sk,num=tablesnum)
    for x in ll:
        if x not in allkeys:
            allkeys.append(x)

# tables: dead

remarks:
* 1st column: rank (-2 weeks, -4 weeks)
* **d(k)**: total dead (in thousand)
* **dd(k)**: last weeks increase in dead (in thousand) 
* **d /1k**: total dead per population (per thousand)
* **dd /1m**: last weeks increase of dead per population (per million)
* **r(%)**: ratechange - relative change of dead last week vs. the week before

In [ ]:
    

dead_table = lambda sk :  ct.get_rank_table_from_df2(mt,mt2,mt4
        , sk,
        ['dead', 'dead_delta','dead_perpop', 'dead_delta_perpop', 'dead_ratechange'], 
        [1/1000,1/1000,1e6, 1e6,100],
        columnalias=['d(k)','dd(k)','d /1m', 'dd /1m', 'r(%)'],
         ascending=False, mininf=tablesmininf,
         num=tablesnum)

for sk in ['dead', 'dead_delta', 'dead_perpop','dead_delta_perpop']:
    display(Markdown("## %s %s"%(sk,lastdayasstr)))
    display(Markdown(dead_table(sk)))
    ll = ct.sort_df_and_get_keys(mt[mt['cases']>tablesmininf],sk,num=tablesnum)
    for x in ll:
        if x not in allkeys:
            allkeys.append(x)


# continents

In [ ]:
for k in ct.list_continents:
    display(Markdown("\n\n### %s"%k))
    if plotsw:
        f, ax1, ax2 = ct.plot_rates(MDlist[0], MDlist[1], k)
        plt.show()
    display(Markdown(ct.get_country_weekly_stat_table(MDlist, pops, k, tableformat='github')))

# countries listed in above tables


In [ ]:
for k in speciallist:
    if k not in allkeys:
        allkeys.append(k)

allkeys.sort()

for k in allkeys:
    display(Markdown("\n\n### %s"%k))
    if plotsw:
        f, ax1, ax2 = ct.plot_rates(MDlist[0], MDlist[1], k)
        plt.show()
    display(Markdown(ct.get_country_weekly_stat_table(MDlist, pops, k, tableformat='github')))